Import packages

In [1]:
import tensorflow as tf
import create_dataset
import math
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

Define generic functions to initialize weight/bias variables, and to define common convolution and pooling operations

In [2]:
def weight_variable(shape):
  initial = tf.truncated_normal(shape, stddev=0.1)
  return tf.Variable(initial, name='weights')

def bias_variable(shape):
  initial = tf.constant(0.1, shape=shape)
  return tf.Variable(initial, name='biases')

In [3]:
def conv2d(x, W):
  return tf.nn.conv2d(x, W, strides=[1, 1, 1, 1], padding='SAME')

def max_pool_2x2(x):
  return tf.nn.max_pool(x, ksize=[1, 2, 2, 1],
                        strides=[1, 2, 2, 1], padding='SAME')

Define the network architecture + loss function and metrics

In [22]:
# Input/Output placeholders
x = tf.placeholder(tf.float32, shape=[None, 121], name='input')
y_ = tf.placeholder(tf.float32, shape=[None, 2], name='output')

num_fc_layers = 10
num_nodes = 121

layers = [x]
keep_prob = tf.placeholder(tf.float32)
prev_layer_size = 121                     
# Create FC layers
for i in range(0, num_fc_layers):
    with tf.name_scope('{}/FC'.format(i)):
        W_fc = weight_variable([prev_layer_size, num_nodes])
        b_fc = bias_variable([121])
        h_fc = tf.nn.sigmoid(tf.matmul(layers[-1], W_fc) + b_fc)
        print layers[-1]
        h_fc_drop = tf.nn.dropout(h_fc, keep_prob)
        layers.append(h_fc_drop)
        prev_layer_size = num_nodes
        

# Readout
with tf.name_scope('5/Readout'):
    W_fc2 = weight_variable([num_nodes, 2])
    b_fc2 = bias_variable([2])
    y_conv = tf.matmul(layers[-1], W_fc2) + b_fc2
    norm_y_conv = tf.nn.l2_normalize(y_conv, dim=0)

# Define loss
l2_loss = tf.reduce_mean(
    tf.nn.l2_loss(tf.subtract(norm_y_conv, y_)) + 0.01*tf.nn.l2_loss(tf.subtract(norm_y_conv, y_conv)))
train_step = tf.train.AdamOptimizer(1e-4).minimize(l2_loss)

Tensor("input_9:0", shape=(?, 121), dtype=float32)
Tensor("0/FC_9/dropout/mul:0", shape=(?, 121), dtype=float32)
Tensor("1/FC_9/dropout/mul:0", shape=(?, 121), dtype=float32)
Tensor("2/FC_9/dropout/mul:0", shape=(?, 121), dtype=float32)
Tensor("3/FC_9/dropout/mul:0", shape=(?, 121), dtype=float32)
Tensor("4/FC_9/dropout/mul:0", shape=(?, 121), dtype=float32)
Tensor("5/FC_9/dropout/mul:0", shape=(?, 121), dtype=float32)
Tensor("6/FC_9/dropout/mul:0", shape=(?, 121), dtype=float32)
Tensor("7/FC_9/dropout/mul:0", shape=(?, 121), dtype=float32)
Tensor("8/FC_9/dropout/mul:0", shape=(?, 121), dtype=float32)


Train the network per-batch

In [20]:
batch_size = 120
logging_frequency = 20
iterations = 800

with tf.Session() as sess:
  sess.run(tf.global_variables_initializer())
  for i in range(iterations):
    batch = create_dataset.generate_data_angles(batch_size)
    if i % logging_frequency == 0:
      loss = sess.run(l2_loss, feed_dict={x: batch[0], y_: batch[1], keep_prob: 1.0}) 
      print('step %d, training loss %g' % (i, loss/batch_size))
    train_step.run(feed_dict={x: batch[0], y_: batch[1], keep_prob: 1.0})

step 0, training loss 0.505392
step 20, training loss 0.505249
step 40, training loss 0.505247
step 60, training loss 0.505247
step 80, training loss 0.505247
step 100, training loss 0.505247
step 120, training loss 0.505247
step 140, training loss 0.505247
step 160, training loss 0.505246
step 180, training loss 0.505246
step 200, training loss 0.505242
step 220, training loss 0.505224
step 240, training loss 0.5031
step 260, training loss 0.503391
step 280, training loss 0.503429
step 300, training loss 0.503418
step 320, training loss 0.503402
step 340, training loss 0.503385
step 360, training loss 0.503369
step 380, training loss 0.503354
step 400, training loss 0.503339
step 420, training loss 0.503324
step 440, training loss 0.50331
step 460, training loss 0.503297
step 480, training loss 0.503283
step 500, training loss 0.50327
step 520, training loss 0.503257
step 540, training loss 0.503243
step 560, training loss 0.503229
step 580, training loss 0.503214
step 600, training l